In [29]:
# Import necessary modules
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from itertools import combinations
from sklearn.preprocessing import TargetEncoder
from sklearn.model_selection import KFold
from sklearn.preprocessing import TargetEncoder

import lightgbm as lgb
import numpy as np


In [28]:
# Load and prepare the data
train_df = pd.read_csv("../data/playground-series-s5e4/train.csv", index_col='id')
test_df = pd.read_csv('../data/playground-series-s5e4/test.csv', index_col='id')
submission_df = pd.read_csv('../data/playground-series-s5e4/sample_submission.csv')

In [21]:
# # Feature engineering function
# ref：https://www.kaggle.com/code/greysky/ps-s5e4-lgbm-cv-12-25-lb-12-15
# def add_features(df):
#
#     # Convert binary features to numeric
#     day_map = {
#         'Monday': 0,
#         'Tuesday': 1,
#         'Wednesday': 2,
#         'Thursday': 3,
#         'Friday': 4,
#         'Saturday': 5,
#         'Sunday': 6,
#     }
#     df['Publication_Day'] = df['Publication_Day'].map(day_map)
#
#     time_map = {
#        "Morning": 0,
#        "Afternoon": 1,
#        "Night": 2
#    }
#
#     df['Publication_Time'] = df['Publication_Time'].map(time_map)
#
#     return df

In [22]:
def add_features(df):
    podc_dict = {'Mystery Matters': 0, 'Joke Junction': 1, 'Study Sessions': 2, 'Digital Digest': 3, 'Mind & Body': 4, 'Fitness First': 5, 'Criminal Minds': 6, 'News Roundup': 7, 'Daily Digest': 8, 'Music Matters': 9, 'Sports Central': 10, 'Melody Mix': 11, 'Game Day': 12, 'Gadget Geek': 13, 'Global News': 14, 'Tech Talks': 15, 'Sport Spot': 16, 'Funny Folks': 17, 'Sports Weekly': 18, 'Business Briefs': 19, 'Tech Trends': 20, 'Innovators': 21, 'Health Hour': 22, 'Comedy Corner': 23, 'Sound Waves': 24, 'Brain Boost': 25, "Athlete's Arena": 26, 'Wellness Wave': 27, 'Style Guide': 28, 'World Watch': 29, 'Humor Hub': 30, 'Money Matters': 31, 'Healthy Living': 32, 'Home & Living': 33, 'Educational Nuggets': 34, 'Market Masters': 35, 'Learning Lab': 36, 'Lifestyle Lounge': 37, 'Crime Chronicles': 38, 'Detective Diaries': 39, 'Life Lessons': 40, 'Current Affairs': 41, 'Finance Focus': 42, 'Laugh Line': 43, 'True Crime Stories': 44, 'Business Insights': 45, 'Fashion Forward': 46, 'Tune Time': 47}
    genr_dict = {'True Crime': 0, 'Comedy': 1, 'Education': 2, 'Technology': 3, 'Health': 4, 'News': 5, 'Music': 6, 'Sports': 7, 'Business': 8, 'Lifestyle': 9}
    week_dict = {'Monday': 0, 'Tuesday': 1, 'Wednesday': 2, 'Thursday': 3, 'Friday': 4, 'Saturday': 5, 'Sunday': 6}
    time_dict = {'Morning': 0, 'Afternoon': 1, 'Evening': 2, 'Night': 3}
    sent_dict = {'Negative': 0, 'Neutral': 1, 'Positive': 2}

    df['Episode_Num'] = df['Episode_Title'].str[8:].astype('category')

    df['Genre'] = df['Genre'].replace(genr_dict)
    df['Podcast_Name'] = df['Podcast_Name'].replace(podc_dict)
    df['Publication_Day'] = df['Publication_Day'].replace(week_dict)
    df['Publication_Time'] = df['Publication_Time'].replace(time_dict)
    df['Episode_Sentiment'] = df['Episode_Sentiment'].replace(sent_dict)

    df['Genre'] = df['Genre'].astype('category')
    df['Podcast_Name'] = df['Podcast_Name'].astype('category')
    df['Publication_Day'] = df['Publication_Day'].astype('category')
    df['Publication_Time'] = df['Publication_Time'].astype('category')
    df['Episode_Sentiment'] = df['Episode_Sentiment'].astype('category')

    df = df.drop(columns=['Episode_Title'])
    return df

In [23]:
# #Apply feature engineering to all datasets
train_df = add_features(train_df)
test_df = add_features(test_df)

/tmp/ipykernel_123312/2953749971.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Genre'] = df['Genre'].replace(genr_dict)
/tmp/ipykernel_123312/2953749971.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Podcast_Name'] = df['Podcast_Name'].replace(podc_dict)
/tmp/ipykernel_123312/2953749971.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, se

In [24]:
encode_columns = ['Episode_Length_minutes', 'Episode_Num', 'Host_Popularity_percentage', 'Number_of_Ads', 'Episode_Sentiment', 'Publication_Day', 'Publication_Time']
pair_size = [2, 3, 4]

for r in pair_size:
    for cols in tqdm(list(combinations(encode_columns, r))):
        new_col_name = '_'.join(cols)

        train_df[new_col_name] = train_df[list(cols)].astype(str).agg('_'.join, axis=1)
        train_df[new_col_name] = train_df[new_col_name].astype('category')

        test_df[new_col_name] = test_df[list(cols)].astype(str).agg('_'.join, axis=1)
        test_df[new_col_name] = test_df[new_col_name].astype('category')

100%|██████████| 35/35 [03:54<00:00,  6.70s/it]


In [27]:
encoded_columns = train_df.columns[11:]
encoder = TargetEncoder(random_state=42)

In [8]:
# Define target column
target = 'Listening_Time_minutes'

In [11]:
# Split data into train and validation sets
train_data, val_data = train_test_split(train_df, test_size=0.2, random_state=42)

In [12]:
train_data

,Podcast_Name,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes,Episode_Num
453635,44,94.30,0,97.67,4,1,NaN,2.0,2,77.27788,81
11651,36,82.15,2,94.78,5,3,NaN,1.0,2,50.02839,53
431999,20,13.72,3,68.60,5,1,65.77,3.0,0,10.07496,21
529211,43,24.00,1,42.14,5,0,41.29,0.0,0,17.82074,99
110925,26,NaN,7,34.10,4,3,NaN,0.0,1,94.80341,15
...,...,...,...,...,...,...,...,...,...,...,...
259178,38,42.65,0,80.53,4,1,96.31,3.0,0,16.77295,71
365838,45,94.50,8,42.80,4,1,8.68,3.0,2,73.00649,66
131932,11,61.54,6,63.97,3,1,62.82,2.0,0,38.48631,18
671155,19,87.86,8,75.76,2,3,74.87,0.0,2,69.27837,27


In [13]:
# Initialize AutoGluon predictor with time constraints
predictor = TabularPredictor(
    label=target,
    problem_type='regression',
    eval_metric='root_mean_squared_error',
    path='ag_models_backpack'
).fit(
    train_data=train_data,
    tuning_data=val_data,
    # Use medium_quality preset instead of best_quality for faster training
    presets='medium_quality',
    # Set a strict 10-minute time limit (600 seconds)
    time_limit=600,
    # Skip hyperparameter tuning to save time
    hyperparameters='default',
    # Limit model types to faster ones
    excluded_model_types=['KNN', 'NN_TORCH', 'FASTAI'],
    verbosity=2
)

print("Done")

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.10.0
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Tue Nov 5 00:21:55 UTC 2024
CPU Count:          12
Memory Avail:       51.36 GB / 57.48 GB (89.4%)
Disk Space Avail:   3550.26 GB / 6519.49 GB (54.5%)
Presets specified: ['medium_quality']
Beginning AutoGluon training ... Time limit = 1200s
AutoGluon will save models to "/mnt/n/code/competition/kaggle/Predict_Podcast_Listening_Time/notebook/ag_models_backpack"
Train Data Rows:    600000
Train Data Columns: 10
Tuning Data Rows:    150000
Tuning Data Columns: 10
Label Column:       Listening_Time_minutes
Problem Type:       regression
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    52533.33 MB
	Train Data (Original)  Memory Usage: 27.19 MB (0.1% of available memory)
	Infe

[1000]	valid_set's rmse: 12.987
[2000]	valid_set's rmse: 12.9599
[3000]	valid_set's rmse: 12.9493
[4000]	valid_set's rmse: 12.9455
[5000]	valid_set's rmse: 12.9464
[6000]	valid_set's rmse: 12.9508


	-12.944	 = Validation score   (-root_mean_squared_error)
	64.18s	 = Training   runtime
	5.31s	 = Validation runtime
Fitting model: LightGBM ... Training model for up to 1128.88s of the 1128.87s of remaining time.


[1000]	valid_set's rmse: 12.9368
[2000]	valid_set's rmse: 12.9136
[3000]	valid_set's rmse: 12.9037
[4000]	valid_set's rmse: 12.9013


	-12.8991	 = Validation score   (-root_mean_squared_error)
	36.62s	 = Training   runtime
	3.37s	 = Validation runtime
Fitting model: RandomForestMSE ... Training model for up to 1088.68s of the 1088.68s of remaining time.
	-12.8035	 = Validation score   (-root_mean_squared_error)
	133.33s	 = Training   runtime
	0.74s	 = Validation runtime
Fitting model: CatBoost ... Training model for up to 949.54s of the 949.53s of remaining time.
	Ran out of time, early stopping on iteration 5177.
	-12.9383	 = Validation score   (-root_mean_squared_error)
	952.68s	 = Training   runtime
	1.16s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ... Training model for up to 360.00s of the -5.09s of remaining time.
	Ensemble Weights: {'RandomForestMSE': 0.579, 'LightGBM': 0.263, 'LightGBMXT': 0.158}
	-12.7292	 = Validation score   (-root_mean_squared_error)
	0.05s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 1205.81s ... Best model: WeightedEnsemble

Done


In [14]:
# # Evaluate on validation data
# performance = predictor.evaluate(val_data)

In [15]:
# performance

{'root_mean_squared_error': -12.729220485024687,
 'mean_squared_error': -162.03305415637212,
 'mean_absolute_error': -9.267427073898412,
 'r2': 0.7797947340895919,
 'pearsonr': 0.883072301499459,
 'median_absolute_error': -6.74962187866211}

In [16]:
# # Generate predictions on test data
# test_pred = predictor.predict(test_df)
# val_pred = predictor.predict(val_data)
# test_pred, val_pred

(id
 750000    55.539909
 750001    18.064295
 750002    49.844246
 750003    76.765488
 750004    47.959488
             ...    
 999995    11.845755
 999996    57.767044
 999997     7.463294
 999998    73.461227
 999999    57.888245
 Name: Listening_Time_minutes, Length: 250000, dtype: float32,
 404846    31.373402
 580313    34.161953
 552086    36.372780
 370876    46.505463
 239330    46.874146
             ...    
 235496    53.114147
 372040    35.043919
 695665     2.838228
 386092    81.920715
 549832    78.157013
 Name: Listening_Time_minutes, Length: 150000, dtype: float32)

In [18]:
# Create submission file
submission = pd.DataFrame({'id': test.index, 'Listening_Time_minutes': test_pred})
submission.set_index('id', inplace=True)
submission.to_csv('../output/submission.csv')  # 12.81 SOTA
print("Submission file created")

Submission file created
